<a href="https://colab.research.google.com/github/Seohee-Kim/KoBERT-Flair-modeling/blob/main/Kobert_0802_2%EC%B0%A8%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 모듈 임포트
* Task1. 필요한 모듈 설치
  * mxnet, tqdm, sentencepiece, transformer, torch
  * 주의: 반드시 transformer 3.0.2 버전이어야 함
  * 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
* Task2. 필요한 모듈 임포트

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 40 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595724 sha256=d080deddb9992218001827a62780893f55765c4d87033950ccad2f91ab8c1616
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.0 MB/s 
     |████████████████████████████████| 769 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
     |████████████████████████████████| 3.0 MB 42.5 MB/s 


In [ ]:
# KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-lqc1g_j3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-lqc1g_j3
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=b6cbe712a77a5791092f4086f880e10d3827025a9080b7dfd88e692b341f0520
  Stored in directory: /tmp/pip-ephem-wheel-cache-hb4hl0ey/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 사용
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


## 데이터 불러오기
* Task1. 코랩 - 드라이브 마운트
* Task2. xlsx 파일 읽고 확인

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
data= pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Data/raw_800.xlsx')

In [ ]:
data

,text,label
0,제가 찾아보니 160까지도 팔던데…,가격
1,그냥.. 30 ㄱ?,가격
2,석가탄신일 전날에 팔게요,날짜
3,몇번 출구 앞인지 알려주셔야…,위치
4,울트라북 10입니다! 실버 색깔이에요,품목
...,...,...
795,좀 비싼거 같은데 5만 7천원으로 해주시면 안될까요,가격
796,아이패드 미니 팔아요 S급 새제품임,품목
797,구두 전 못신겠더라구요…,품목
798,테팔 프라이팬 세트 팜 3개 들어있음,품목


## 데이터 전처리
* Task1. label 정수 인코딩
  * e.g. 날짜 -> 0, 위치 -> 1 ...
* Task2. data_list 생성: ['text', 'label'] 형식으로 변경
  * cf. 현재는 비교적 클린 데이터라서 불필요하지만, 후에 불용어 제거 등의 단계가 수반될 수 있음

In [ ]:
# 라벨 확인
data['label'].unique()

array(['가격', '날짜', '위치', '품목'], dtype=object)

In [ ]:
data.loc[(data['label'] == "날짜"), 'label'] = 0
data.loc[(data['label'] == "위치"), 'label'] = 1
data.loc[(data['label'] == "품목"), 'label'] = 2
data.loc[(data['label'] == "가격"), 'label'] = 3

In [ ]:
# 바뀐 라벨 확인
data['label'].unique()

array([3, 0, 1, 2], dtype=object)

In [ ]:
data_list = []
for q, label in zip(data['text'], data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[223])
print(data_list[300])
print(data_list[500])

['강창교에 소방서하나 있어서 거기서 보죠', '1']
['몇월 며칠에 볼까?', '0']
['S급 콜트 일렉기타 팔게요 기스 하나도 없어요', '2']


## Train : Test Data
* 트레인 : 테스트 데이터 분리 (8:2)

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=999)

In [ ]:
print('Train : Test 데이터 개수')
print(str(len(dataset_train)) + ':' + str(len(dataset_test)))

Train : Test 데이터 개수
640:160


## Text -> KoBert 입력 데이터 변환
* 토큰화, 패딩
  * BERTSentenceTransform 모듈을 사용
* 하이퍼 파라미터 설정
  * max_len: 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음.
* Torch 형식의 데이터셋 생성

In [ ]:
 class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
# 잘 실행되었나?
# 잘 실행될 경우 3개의 어레이가 출력됨
#   - 첫 번째는 패딩된 시퀀스
#   - 두 번째는 길이와 타입에 대한 내용
#   - 세 번재는 어텐션 마스크 시퀀스

data_train[0]

(array([   2, 2013, 7605,  611, 5495, 7362, 4814, 6797, 6999,  517,    5,
        2680,  517, 6186, 7374, 7843,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(17, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## KoBert 학습 모델 생성
* init > num_classes = n : 라벨 숫자 기입

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer, scheduler
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도(Accuracy)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1) # 예측 결과 중 최댓값과 해당 index 추출
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

## 모델 학습
* 에포크 = 5 (위에서 설정했던 하이퍼 파라미터만큼)

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0: # epoch 마다 train accuracy 출력
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    # 평가 모드
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


KeyboardInterrupt: ignored

## New Data TEST
* 새로운 문장에 대한 예측 정확도 확인

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("날짜")
            elif np.argmax(logits) == 1:
                test_eval.append("위치")
            elif np.argmax(logits) == 2:
                test_eval.append("품목")
            elif np.argmax(logits) == 3:
                test_eval.append("가격")

        real_label = input("실제 라벨을 입력하세요: ")

        print(">> 예측 라벨 정보:", test_eval[0])
        print(">> 실제 라벨 정보:", real_label)
        if test_eval[0] == real_label:
          print('라벨 예측에 성공하였습니다.')
        else:
          print('라벨 예측에 실패하였습니다.')
        print(">> 예측 결과 정보 : \n", out)
        print("------------------------------------------------------------")
        print(" ")

using cached model


In [ ]:
while True:
  sentence = input("테스트할 문장을 입력하세요: ")
  if (sentence == "."):
    break
  predict(sentence)

테스트할 문장을 입력하세요: 네네


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


실제 라벨을 입력하세요: 모름
>> 예측 라벨 정보: 날짜
>> 실제 라벨 정보: 모름
라벨 예측에 실패하였습니다.
>> 예측 결과 정보 : 
 tensor([[ 1.8569,  1.1158, -1.9020, -1.1091]], grad_fn=<AddmmBackward>)
------------------------------------------------------------
 
테스트할 문장을 입력하세요: .


# 여기

In [ ]:
PATH = "/content/drive/MyDrive/Colab Notebooks/kobertModel_v0728.pt"
model = torch.load(PATH, map_location=torch.device('cpu'))
model.eval()

## 문장만 sorting

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 날짜, 품목, 위치, 가격

def predict2(sent):
    data = [sent, '0']
    dataset_another = [data]
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_eval=[]
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()
            if np.argmax(logits) == 0:
                test_eval.append("날짜")
            elif np.argmax(logits) == 1:
                test_eval.append("위치")
            elif np.argmax(logits) == 2:
                test_eval.append("품목")
            elif np.argmax(logits) == 3:
                test_eval.append("가격")
        return test_eval[0], out

def sorting(sent):
  date_max = -1
  item_max = -1
  place_max = -1
  price_max = -1

  a = ""
  b = ""
  c = ""
  d = ""
  for text in sent:
    test_eval, out = predict2(text)
    max_val, max_index = torch.max(out, 1)
    ##print(text)
    ##print(max_val)
    ##print("\n")

    if test_eval == '날짜':
      if date_max < max_val:
        a = text
        date_max = max_val
    elif test_eval=='품목':
      if item_max < max_val:
        b = text
        item_max = max_val
    elif test_eval =='위치':
      if place_max < max_val:
        c = text
        place_max = max_val
    elif test_eval =='가격':
      if price_max < max_val:
        d = text
        price_max = max_val


  print(
    '날짜 정보 : ', a,
    '\n품목 정보 : ', b,
    '\n위치 정보 : ', c,
    '\n가격 정보 : ', d)

using cached model


In [ ]:
sent = ['안녕하세요', '네 안녕하세요', '물건 파시는 분 맞으시죠', '네네 자전거 사고싶어용', '서울역에서 보시죠', '좋아요', '네', '얼마죠?', '6만원입니다', '1월 2일에 거래 어떠세요?', '좋아요 그때 뵈요']
sorting(sent)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


날짜 정보 :  1월 2일에 거래 어떠세요? 
품목 정보 :  네네 자전거 사고싶어용 
위치 정보 :  서울역에서 보시죠 
가격 정보 :  6만원입니다


## 코모란

In [ ]:
!pip install konlpy
!pip install PyKomoran

In [ ]:
from konlpy.tag import Komoran
komoran = Komoran()

#from PyKomoran import Komoran, DEFAULT_MODEL
#komoran = Komoran(DEFAULT_MODEL['LIGHT'])

In [ ]:
# 테스트
# komoran.get_plain_text('1월 2일에 거래 어떠세요?')
# komoran.get_morphes_by_tags('1월 2일에 거래 어떠세요?', tag_list=['SN','NNB'])

In [ ]:
# user_dic
# 참고 자료: https://github.com/shineware/PyKOMORAN/issues/49
komoran = Komoran(userdic='/content/drive/MyDrive/Colab Notebooks/Data/user_dic.txt')

## 문장에서 단어만 sorting

In [ ]:
class ClassifierModule():


  def __init__(self):
    self.date_sent = ""
    self.item_sent = ""
    self.place_sent = ""
    self.price_sent = ""

    self.words = []

    self.date_word = ""
    self.item_word = ""
    self.place_word = ""
    self.price_word = ""


  def sentence_to_words(self):
    self.words = komoran.nouns(self.date_sent) + komoran.nouns(self.item_sent) + komoran.nouns(self.place_sent)
    #print(self.words)

  def sorting(self, text):
    date_max = -1
    item_max = -1
    place_max = -1
    price_max = -1

    for sent in text:
      test_eval, out = predict2(sent)
      max_val, max_index = torch.max(out, 1)
      #print(sent)
      #print(max_val)
      #print("\n")

      if test_eval == '날짜':
        if date_max < max_val:
          self.date_sent = sent
          date_max = max_val
      elif test_eval=='품목':
        if item_max < max_val:
          item_max = max_val
          self.item_sent = sent
      elif test_eval =='위치':
        if place_max < max_val:
          place_max = max_val
          self.place_sent = sent
      elif test_eval =='가격':
        if price_max < max_val:
          regex = re.compile("\d+(원|만원|마넌)|\d+만|만원|\d.\d(만원|만)?")
          result = regex.search(sent)
          if result:
            self.price_sent = result.group(0)


    print(
      '>> 문장만 Sorting <<',
      '\n날짜 정보 : ', self.date_sent,
      '\n품목 정보 : ', self.item_sent,
      '\n위치 정보 : ', self.place_sent,
      '\n가격 정보 : ', self.price_sent)

  def words_sorting(self, text):
    self.sorting(text)
    self.sentence_to_words()
    date_max = -1
    item_max = -1
    place_max = -1
    price_max = -1
    for word in self.words:
      test_eval, out = predict2(word)
      max_val, max_index = torch.max(out, 1)
      #print(word)
      #print(max_val)
      #print("\n")

      if test_eval == '날짜':
        if date_max < max_val:
          self.date_word = word
          date_max = max_val
      elif test_eval=='품목':
        if item_max < max_val:
          self.item_word = word
          item_max = max_val
      elif test_eval =='위치':
        if place_max < max_val:
          self.place_word = word
          place_max = max_val

    self.price_word = self.price_sent

    print(' ')
    print(
      '>> 핵심 단어만 Sorting <<',
      '\n날짜 정보 : ', self.date_word,
      '\n품목 정보 : ', self.item_word,
      '\n위치 정보 : ', self.place_word,
      '\n가격 정보 : ', self.price_word)

In [ ]:
Cm = ClassifierModule()

In [ ]:
sent1 = ['안녕하세요', '네 안녕하세요', '물건 파시는 분 맞으시죠', '네네 자전거 사고싶어용', '서울역에서 보시죠', '좋아요', '네', '얼마죠?', '6만원입니다', '1월 2일에 거래 어떠세요?', '좋아요 그때 뵈요']
Cm.words_sorting(sent1)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 문장만 Sorting << 
날짜 정보 :  1월 2일에 거래 어떠세요? 
품목 정보 :  네네 자전거 사고싶어용 
위치 정보 :  서울역에서 보시죠 
가격 정보 :  6만원
 
>> 핵심 단어만 Sorting << 
날짜 정보 :  1월 2일 
품목 정보 :  자전거 
위치 정보 :  서울역 
가격 정보 :  6만원


In [ ]:
sent2 = ['안뇽하세욤', '넴넴', '저 자전거 거래할려구요', '7만원이요.','강남역 패스트파이브에서 봐용', '넴 다음주 어때요', '좋습니다']
Cm.words_sorting(sent2)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 문장만 Sorting << 
날짜 정보 :  넴 다음주 어때요 
품목 정보 :  저 자전거 거래할려구요 
위치 정보 :  강남역 패스트파이브에서 봐용 
가격 정보 :  7만원
 
>> 핵심 단어만 Sorting << 
날짜 정보 :  다음주 
품목 정보 :  자전거 
위치 정보 :  강남역 
가격 정보 :  7만원


In [ ]:
sent3 = ['8월 15일에 부산역 앞에서 거래하고 싶어요', '좋습니다', '냉장고입니다.', '100만원에 하시죠' ,'하하', '80만원에 해줘', '안돼요 100만원으로 결정합시다']
CM3 = ClassifierModule()
CM3.words_sorting(sent3)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 문장만 Sorting << 
날짜 정보 :  8월 15일에 부산역 앞에서 거래하고 싶어요 
품목 정보 :  냉장고입니다. 
위치 정보 :   
가격 정보 :  100만원
 
>> 핵심 단어만 Sorting << 
날짜 정보 :  8월 15일 
품목 정보 :  냉장고 
위치 정보 :  부산역 
가격 정보 :  100만원


## KSS

### 문장 분리란
* '안녕하세요 저는 김서희입니다' --> '안녕하세요', '저는 김서희입니다'
  
### 사용할 수 있는 라이브러리 kss
* 특징: 패턴 기반 문장분리기
* 장점: 속도 빠름
* 단점: "패턴 기반" (종결형 음절에 대한 구분으로 센텐스 바운더리 지정)으로 정확도 떨어짐
  * 예시) 안녕하세요 저는 김서희입니다 -> '요'가 있기 때문에 분리 가능
  * 예시) 난 그렇게 생각안해 차라리 3만원이 나음 -> 분리 불가능
  
#### 현재 분류 방식에 적용할 때
* 현재 방식
  * 1. 각 라벨에 대해 가장 높은 확률을 가지는 "문장" 선택 (1차 분류)
  * 2. 각 라벨에 대해 가장 높은 확률을 가지는 "명사" 선택 (2차 분류)
* 따라서 문장 분리로 리스트의 개수가 늘어나면 확률 경쟁시 연산해야할 것이 많아짐
* 연산 속도가 저하되진 않을 것, 정확도?

In [ ]:
!pip install kss

     |████████████████████████████████| 65 kB 2.2 MB/s 


In [ ]:
import kss

In [ ]:
s = "안녕하세요 저는 김서희입니다"
for sent in kss.split_sentences(s):
  print(sent)

안녕하세요
저는 김서희입니다


In [ ]:
s = "난 그렇게 생각안해 차라리 3만원이 나음"
for sent in kss.split_sentences(s):
  print(sent)

난 그렇게 생각안해 차라리 3만원이 나음
